In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

# Paper 1 - Uncostrained model 

## Generate synthetic data  
Assume data is generated from the true model  
$$
    Y_i = \mu + \int_{T} x(t)\beta(t) \,dt + \epsilon_i
$$
$$
    \beta(t) = sin(t*\pi/4) \text{ true parameter}
$$
$$
        x(t) = GP(0, K) \text{ synthesized covariate}
$$
$$
    \epsilon_i \sim N(0,\sigma^2)
$$

In [2]:
def integrate(T, f): 
    tot = 0;
    for i in range(len(T)-1):
        tot = tot + (1/2 * (f[i]+f[i+1]) * (T[1]-T[0]))
    return tot

In [3]:
class data_generator:
    def __init__(self, T1, T2, g, beta, n, covariates_kernel, sigma):
        self.T = np.linspace(T1,T2,g)
        self.K = np.fromfunction(np.vectorize(lambda s, t: covariates_kernel(self.T[s], self.T[t])), (g, g), dtype=int)
        self.g = g
        self.n = n
        self.beta = beta
        self.sigma = sigma
    
    def grid(self):
        return self.T
    
    def i_o(self):
        x = np.random.multivariate_normal(np.zeros(self.g), self.K, self.n)
        y = np.fromfunction(np.vectorize(lambda i: integrate(self.T,  x[i,:]*beta)+np.random.normal(0,self.sigma,1)), (self.n,), dtype=int)
        return x,y
        

In [34]:
# EXAMPLE
# input
a = -3
b = 3
g = 2**7
n = 100
sigma = 1
beta = np.sin(np.linspace(a,b,g)*np.pi/4)
def covariates_kernel(s,t):
    return np.exp(-2*(t-s)**2)
def prior_kernel(s,t):
    return np.exp(-(t-s)**2)

#generation
data = data_generator(b,a,g,beta,n,covariates_kernel, sigma)
T = data.grid()
x,y = data.i_o()

## Compute Posterior

Assume the folowing model for the data
$$
    Y_i|\beta \sim N(\int_{T} x_i(t)*\beta(t) dt, \sigma^2)
$$
$$
    \beta \sim GP(0,K)
$$
The posterior distribution for $\beta$ is
$$
    \beta|Y \sim GP(m,K^*)
$$
$$
    m(t) = Lx(t)^{'}(\Sigma + \sigma^2I_n)^{-1}Y
$$
$$
    K^*(s,t) = K(s,t) - Lx(s)^{'}(\Sigma + \sigma^2I_n)^{-1}Lx(t)
$$
with
$$
    Lx(t)^{'} = (Lx_1(t), ..., Lx_n(t))
$$
$$
  Lx_i(t) = \int_{T} K(t,s)x_i(s)ds  
$$
$$
    Y = (Y_1,...,Y_n)^{'}
$$
$$
    \Sigma_{ij} = R(x_i, x_j)
$$
$$
    R(x_i, x_j) = \int_{T} \int_{T} K(s,t)*x_i(s)x_j(t)ds dt
$$

In [11]:
class posterior:
    def __init__(self, T, x, y, prior_kernel,sigma):
        self.T = T
        self.x = x
        self.y = y
        self.prior_kernel = prior_kernel
        self.n = y.shape[0]
        self.g = T.shape[0]
        self.sigma = sigma
        self.K = np.fromfunction(np.vectorize(lambda s, t: self.prior_kernel(self.T[s], self.T[t])), (self.g, self.g), dtype=int)
    
    def Lx(self,t):   
        L = np.zeros(self.n)    
        for i in np.arange(0,self.n):
            L[i] = integrate(self.T,self.K[t, :] * self.x[i,:])
        return L
    
    def R(self,i, j):
        Ri = np.fromfunction(np.vectorize(lambda t: integrate(self.T,self.K[:,t] * self.x[i,:])), (self.g,), dtype=int)
        Rij = integrate(self.T, Ri * self.x[j,:])
        return  Rij
    
    def cov(self):
        return np.fromfunction(np.vectorize(lambda i, j: self.R(i, j)), (self.n, self.n), dtype=int)
    
    def posterior_mean_cov(self):
        inv = np.linalg.inv(self.cov() + self.sigma * np.identity(self.n))
        def m(t):
            mean = np.dot(np.dot(np.transpose(self.Lx(t)),inv),self.y)
            return mean
        def Kstar(s,t):
            cov = self.K[s,t] - np.dot(np.dot(np.transpose(self.Lx(s)),inv),self.Lx(t))
            return cov
        
        m = np.fromfunction(np.vectorize(lambda t: m(t)), (self.g,), dtype=int)
        C = np.fromfunction(np.vectorize(lambda t, s: Kstar(s,t)), (self.g,self.g), dtype=int)
        return m,C
    
    def only_mean(self):
        inv = np.linalg.inv(self.cov() + self.sigma * np.identity(self.n))
        def m(t):
            mean = np.dot(np.dot(np.transpose(self.Lx(t)),inv),self.y)
            return mean
        m = np.fromfunction(np.vectorize(lambda t: m(t)), (self.g,), dtype=int)
        return m
    
    def fitted_values(self):
        m = self.only_mean()
        return np.fromfunction(np.vectorize(lambda i: integrate(T, self.x[i,:]*m)), (self.n,), dtype=int)


In [ ]:
# EXAMPLE
p = posterior(T,x,y,prior_kernel,sigma)
m,cov = p.posterior_mean_cov()
fitted = p.fitted_values()

## Simulations and Fitted Values 

In [ ]:
samples = np.random.multivariate_normal(m, cov, size=1000)

up = np.zeros(m.shape[0])
down = np.zeros(m.shape[0])
for i in range(m.shape[0]):
    mean, var, std = scipy.stats.bayes_mvs(samples[:,i], alpha=0.95)
    mean_estimate, std_estimate = mean.statistic, std.statistic
    down[i],up[i] = mean_estimate - std_estimate, mean_estimate + std_estimate 



plt.plot(T,m, label='Posterior Mean')
plt.fill_between(T, down, up, alpha=0.3, label='95% Credibility Interval')

plt.title('Posterior Mean with 95% Credibility Interval')
plt.xlabel('Parameter Value')
plt.legend()
plt.show()

In [ ]:
plt.scatter(y,fitted)
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')